<a href="https://colab.research.google.com/github/ProfAI/nlp00/blob/master/10%20-%20Seq2Seq%20e%20Machine%20Translation/machine_translation_char.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Architetture Sequence 2 Sequence per la traduzione automatica
In questo notebook creeremo un'architettura **Sequence 2 Sequence** di rete neurale artificiale in grado di tradurre del testo dall'italiano all'inglese, per farlo avremo bisogno di un corpus di testo contenente frasi di esempio in entrambe le lingue, fortunatamente il software di flashcard Anki ci mette a disposizione tali corpus per molteplici lingue, scarichiamo quello per inglese-italiano da [questo link](http://www.manythings.org/anki/). Se usi Google colab o hai wget installato, scarica pure il file zip eseguendo la cella di codice qui sotto.

In [1]:
!wget http://www.manythings.org/anki/ita-eng.zip

--2019-05-02 10:52:51--  http://www.manythings.org/anki/ita-eng.zip
Resolving www.manythings.org (www.manythings.org)... 104.24.109.196, 104.24.108.196, 2606:4700:30::6818:6cc4, ...
Connecting to www.manythings.org (www.manythings.org)|104.24.109.196|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3981147 (3.8M) [application/zip]
Saving to: ‘ita-eng.zip’

ita-eng.zip         100%[===================>]   3.80M  14.8MB/s    in 0.3s    

2019-05-02 10:52:51 (14.8 MB/s) - ‘ita-eng.zip’ saved [3981147/3981147]



ed estrai lo zip

In [2]:
!unzip ita-eng.zip

Archive:  ita-eng.zip
  inflating: ita.txt                 
  inflating: _about.txt              


Adesso abbiamo il file ita.txt contenente coppie di frasi in inglese e italiano su ogni riga, separate da un tab. Dividiamo le frasi in base alla lingua in due liste separate e salviamo ogni carattere all'interno di un set, mentre scorriamo il file processiamo anche le frasi, rimuovendo la punteggiatura e convertendo tutto in minuscolo.

In [3]:
import re

en_chars = set({})
it_chars = set({})

it_sents = []
en_sents = []

with open("ita.txt") as en_it_sents:
  lines = en_it_sents.read().split("\n")
  
  num_samples = 100000
  
  for line in lines[:min(num_samples, len(lines)-1)]:
    
    line = re.sub(r'[^\w\s]','',line)
    line = line.lower()
    
    en_sent, it_sent = line.split("\t")
    en_sent = "\t"+en_sent+"\n"
    
    en_sents.append(en_sent)
    it_sents.append(it_sent)
    
    for char in en_sent:
      en_chars.add(char)
      
    for char in it_sent:
      it_chars.add(char)
      
print("Numero di frasi di esempio %d" % num_samples)

Numero di frasi di esempio 100000


Convertiamo i set di caratteri in liste ordinate e contiamo il numer do caratteri contenuti in ogni lista.

In [4]:
it_chars = sorted(list(it_chars))
en_chars = sorted(list(en_chars))

num_encoder_tokens = len(it_chars)
num_decoder_tokens = len(en_chars)

print('Numbero di token di input:', num_encoder_tokens)
print('Number di token di output:', num_decoder_tokens)

Numbero di token di input: 46
Number di token di output: 41


Contiamo la lungezza massima di una frase per entrambe le lingue.

In [5]:
max_encoder_seq_length = max([len(sent) for sent in it_sents])
max_decoder_seq_length = max([len(sent) for sent in en_sents])

print("Lunghezza massima della sequenza per l'input:", max_encoder_seq_length)
print("Lunghezza massima della sequenza per l'output:", max_decoder_seq_length)

Lunghezza massima della sequenza per l'input: 100
Lunghezza massima della sequenza per l'output: 23


Adesso dobbiamo eseguire il one hot encoding di ogni frase a livello di carattere, per farlo ci conviene creare il dizionario che ci permette di accedere velocemente all'indice partendo dal carattere.

In [0]:
it_token_index = dict(
    [(char, i) for i, char in enumerate(it_chars)])
en_token_index = dict(
    [(char, i) for i, char in enumerate(en_chars)])

Eseguiamo il one hot encoding per creare:
- **input dell'encodere**: le frasi italiane.
- **input del decoder**: le frasi in inglese.
- **output del decoder**: le frasi in inglese shiftate di un carattere.

In [0]:
import numpy as np

# inizializziamo gli array vuoti

encoder_input_data = np.zeros((len(it_sents), max_encoder_seq_length, num_encoder_tokens))
decoder_input_data = np.zeros((len(it_sents), max_decoder_seq_length, num_decoder_tokens))
decoder_target_data = np.zeros((len(it_sents), max_decoder_seq_length, num_decoder_tokens))

# iteriamo simultaneamente su frasi in italiano e inglese

for i, (it_sent, en_sent) in enumerate(zip(it_sents, en_sents)):
    for t, char in enumerate(it_sent):
        # assegnamo un 1 all'indice di ogni carattere contenuto nella frase
        encoder_input_data[i, t, it_token_index[char]] = 1.
    for t, char in enumerate(en_sent):
        # assegnamo un 1 all'indice di ogni carattere contenuto nella frase
        decoder_input_data[i, t, en_token_index[char]] = 1.
        if t > 0:
            # shiftiamo di uno il target
            decoder_target_data[i, t - 1, en_token_index[char]] = 1.

I dati per l'addestramento sono pronti, possiamo passare alla creazione del modello.

## Creazione del Modello
Il modello che andremo a creare è molto complesso e alcuni dei suoi strati devono accettare input da strati differenti (il decoder prende lo stato dall'encoder e come input le frasi in inglese), dobbiamo utilizzare le [API Funzionali di Keras](https://keras.io/getting-started/functional-api-guide/). 
<br>
Cominciamo con l'**encoder**:
- Usiamo la classe **Input** per definire l'input dell'encoder.
- Creiamo lo strato ricorrente (LSTM) dell'encoder, che dovrà restituire lo stato (return_state=True)
- Usiamo l'encoder, scartiamo l'ouput (primo array ritornato) e teniamo solo lo stato (secondo e terzo array)


In [8]:
from keras.layers import Input, LSTM, Dense

encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(256, return_state=True)
_, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


Adesso creiamo il **decoder**:
- Usiamo la classe **Input** per definire l'input dell decoder.
- Creiamo lo strato ricorrente (LSTM) dell'encoder, che dovrà usare lo stato dell'encoder (initial_state=encoder_states)
- Usiamo l'encoder, scartiamo l'ouput (primo array ritornato) e teniamo solo lo stato (secondo e terzo array).
- Creiamo uno strato di output che utilizzerà la funzine di attivazione softmax per eseguire una classificazione multiclasse.


In [0]:
decoder_inputs = Input(shape=(None, num_decoder_tokens))

decoder_lstm = LSTM(256, return_sequences=True, return_state=True)

decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)

decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

Usiamo la classe **Model** di keras per creare il modello, passando gli strati di input (sia quello dell'encoder che quello del decoder) all'interno di una lista e lo strato di output.

In [0]:
from keras.models import Model

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

Compiliamo il modello, usiamo come algoritmo di ottimizzazione 'rmsprop', il quale dovrebbe portare risultati migliori quando si lavora con modelli sequenziali.

In [0]:
model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=["accuracy"])

Ed avviamo l'addestramento, qui dobbiamo passare sempre i dati di input, sia per l'encoder che per il decoder, all'interno di una lista e i dati di output.
<br><br>
**NOTA BENE** 
<br>
Se non hai una GPU che supporta la tecnologia CUDA e non vuoi usare Google Colaboratory, ti consiglio di importare il modello pre-addestrato eseguendo il codice nella cella poco più in basso, altrimenti l'addestramento potrebbe richiedere anche giorni e mettere sotto forte stress il tuo pc.


In [32]:
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=256,
          epochs=1,
          validation_split=0.2)

Train on 80000 samples, validate on 20000 samples
Epoch 1/1
80000/80000 [==============================] - 50s 629us/step - loss: 0.1153 - acc: 0.6741 - val_loss: 2.4167 - val_acc: 0.6053


Se stai usando una GPU che supporta la tecnologia CUDA, sul tuo computer o con Google Colaboratory, l'addestramento per 500 epoche dovrebbe richiedere un paio di ore, se non vuoi aspettare puoi ridurre il numero di epoche a non meno di 100 oppure importare il modello che ho già addestrato eseguendo il codice qui sotto.

In [13]:
from urllib.request import urlretrieve
from keras.models import load_model

model_file = "translate_500.h5"
model_path = "https://github.com/ProfAI/nlp00/raw/master/10%20-%20Seq2Seq%20e%20Machine%20Translation/model/translate_500.h5"

urlretrieve(model_path, model_file)

model = load_model(model_file)
model.evaluate([encoder_input_data, decoder_input_data], decoder_target_data)

100000/100000 [==============================] - 118s 1ms/step


[0.5588066027265787, 0.6640221741580963]

## Usare il modello per tradurre del testo
Per calcolare l'output della rete, cioè la frase tradotta, dobbiamo dividere il modello addestrato in due modelli, uno per l'encoder

In [0]:
encoder_model = Model(encoder_inputs, encoder_states)

ed uno per il decoder

In [0]:
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))

decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]

decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)

decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)

decoder_model = Model([decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states)

Definiamo una funzione che ci permette di eseguire il one hot encoding dell'input.

In [0]:
def encode(text):
  
  text_encoded = np.zeros((max_encoder_seq_length, num_encoder_tokens))

  for c, char in enumerate(text):
        text_encoded[c, it_token_index[char]] = 1.
      
  return np.array([text_encoded])

In [0]:
en_index_token = dict(
    (i, char) for char, i in en_token_index.items())

Adesso gli step per eseguire la predizione sono i seguenti:
1. Codifichiamo la frase da tradurre utilizzando il one hot encoding.
2. Utilizziamo l'encoder per ottenere lo stato iniziale
3. Creiamo una sequenza da dare come input iniziale al decoder, contente soltanto il carattere di inizio sequenza /t.
4. Utilizziamo il decoder per ottenere il carattere successivo della sequenza e i nuovi stati.
5. Aggiungiamo il carattere alla sequenza.
6. Ripetiamo gli step 4 e 5 fino a quando non incontriamo il carattere di fine sequenza /n o fino a quando la lunghezza della sequenza non è maggiore alla massima consetita. 

In [0]:
def translate(text):
  
    text = encode(text)
  
    # Otteniamo lo stato inziale usando l'encoder
    states_value = encoder_model.predict(text)

    # Creiamo un'array vuoto per l'input del decoder
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Inseriamo in prima posizione il carattere di inizio sequenza
    target_seq[0, 0, en_token_index['\t']] = 1.

    stop = False
    decoded_sentence = ""
    while not stop:
        
        # Usiamo il decoder per predire il seguente carattere della sequenza
        # e i nuovi stati
      
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # Estraiamo il carattere usando l'indice predetto
        # e aggiungiamolo alla sequenza
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = en_index_token[sampled_token_index]
        decoded_sentence += sampled_char

        # Se abbiamo incontrato il carattere di fine sequenza
        # o se la lunghezza della sequenza è maggiore della lunghezza massima
        # interrompiamo
        if (sampled_char == '\n' or len(decoded_sentence) > max_decoder_seq_length):
            stop = True

        # Aggiorniamo l'input del decoder
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Aggiorniamo gli stati
        states_value = [h, c]

    return decoded_sentence

E' tutto pronto ! Proviamo ad utilizzare il nostro traduttore.

In [2]:
print(translate('hi'))

NameError: ignored

In [0]:
text_it = None

while text_it != "ciao":
   
  text_it = input("Italiano: ")
  text_en = translate(text_it)

  print('English:', text_en)

Italiano: come stai
English: how is the sald

Italiano: ciao
English: he should thank me

